# CS-UY 3943 - Introduction of Data Science
## Final Project: Tweet Sentiment Extraction

Please edit the cell below to add your name, NetID, assigned group, and group members for this assignment. **Complete this section before proceeding to the assignment questions.**

Name:  Taylor Wong
NetID:  tw1835
Group Member 1:  Thomas Scarola
NetID:  tws285
***

### Preliminary Data Analysis

#### Include 1 or 2 plots or data tables that demonstrate initial EDA efforts


1\. Read the data file `train.csv` into your program as a `pandas` `DataFrame`

In [94]:
#Place your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

train_df = pd.read_csv('train.csv')
train_df.head(20).sort_values('Survived')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


<hr />

2a. Figure out who survived: which column had highest correlation with `Survived`?
<br><br>
Method A: find highest correlation between data as is (only numerical/easy data)


In [43]:
# Figure out who survived: which column had highest correlation with survival
highest_correlation_coeff = 0
highest_correlation_col = 'Survived'

for col in train_df.columns:
    try:
        if col != 'Survived':
            curr_coeff = np.corrcoef(train_df['Survived'], train_df[col])[0,1]
            if curr_coeff > highest_correlation_coeff:
                highest_correlation_coeff = curr_coeff
                highest_correlation_col = col
    except:
        pass

print(highest_correlation_coeff)
print(highest_correlation_col)

0.25730652238496227
Fare


In [44]:
SurvivedVsFare20 = train_df[['Survived', 'Fare']].head(20).sort_values('Survived')
SurvivedVsFare20Top = SurvivedVsFare20.head(10)
SurvivedVsFare20Bottom = SurvivedVsFare20.tail(10)
(np.mean(SurvivedVsFare20Top['Fare']), np.mean(SurvivedVsFare20Bottom['Fare']))

(19.1, 25.298740000000002)

<hr />

2b. Figure out who survived: which column had highest correlation with `Survived`?
<br><br>
Method B: first separate into dummy classes to access all categorical data too, then do correlations


In [91]:
# Figure out who survived: rank columns in order of correlation with survival

# convert categorical data to dummies
train_df_origin = pd.get_dummies(train_df.drop(columns='Name'))
# train_df_origin.drop(columns=['Sex_male', 'Embarked_S'])
print(train_df_origin.head())

# figure out correlation between survival and all other features
correlationWithSurvival = []
for col in train_df_origin.columns:
    correlationWithSurvival.append(np.corrcoef(train_df_origin['Survived'], train_df_origin[col])[0,1])

# create correlation with survival dictionary 
cws_df = pd.DataFrame({'col_name': train_df_origin.columns, 'cws': correlationWithSurvival})
cws_df.sort_values('cws', ascending=False)

   PassengerId  Survived  Pclass   Age  SibSp  Parch     Fare  Sex_female  \
0            1         0       3  22.0      1      0   7.2500           0   
1            2         1       1  38.0      1      0  71.2833           1   
2            3         1       3  26.0      0      0   7.9250           1   
3            4         1       1  35.0      1      0  53.1000           1   
4            5         0       3  35.0      0      0   8.0500           0   

   Sex_male  Ticket_110152  ...  Cabin_F G73  Cabin_F2  Cabin_F33  Cabin_F38  \
0         1              0  ...            0         0          0          0   
1         0              0  ...            0         0          0          0   
2         0              0  ...            0         0          0          0   
3         0              0  ...            0         0          0          0   
4         1              0  ...            0         0          0          0   

   Cabin_F4  Cabin_G6  Cabin_T  Embarked_C  Embarked_Q  

,col_name,cws
1,Survived,1.000000
7,Sex_female,0.543351
6,Fare,0.257307
837,Embarked_C,0.168240
42,Ticket_113760,0.085083
...,...,...
577,Ticket_CA. 2343,-0.070234
839,Embarked_S,-0.155660
2,Pclass,-0.338481
8,Sex_male,-0.543351


<hr>

#### Brief discussion of what EDA has revealed about the data

<p>
<span style="color:blue">
Gender and fare were the first and second most influential factors in determining who would survive, respectively. Particularly, females and the wealthy were more likely to survive.
</span>
</p>